In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#loading the dataa
fake=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")
true=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")

In [ ]:
#checking how the data looks like
fake.head(10)
fake.tail(10)

In [ ]:
#checking different subjects and thier counts - like number of topics under that subject
fake["subject"].value_counts()
#so this value_counts() helps to count !!

In [ ]:
true.head(10)

In [ ]:
true.subject.value_counts()

In [ ]:
#creating a target column ( category ) for whether fake or not
fake['category']=1
true['category']=0

In [ ]:
#joining the data the two data frames ( true and false) and reseting the index
df=pd.concat([fake, true]).reset_index(drop=True)

In [ ]:
df.head()
# reperesentation of the conacted data if you do tail you can see 0 category true data too.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

plt.figure(figsize=(6,7))
sns.countplot(df["category"])
plt.show()

In [ ]:
print(df["category"].value_counts())

In [ ]:
plt.figure(figsize=(6, 7))
ax = sns.countplot(x="category", data=df, order=[0, 1])  # Manually specify categories
plt.title("Distribution of Categories")
plt.xlabel("Category (0 = Real, 1 = Fake)")
plt.ylabel("Count")

# Add counts on top of bars
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                xytext=(0, 5), 
                textcoords='offset points')

plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
ax = sns.countplot(x="subject", data=df)
plt.show()

In [ ]:
df=df[['text','category']]

In [ ]:
df.tail()

**DATA CLEANIGN FROM HERE !!**
in this we find the missing value with isna() func. and then clean it accordingly.

In [ ]:
df.isna().sum()*100/len(df)

#this *100/len(df) is to have the missing values in percentage %

In [ ]:
# Checking for empty strings or whitespace-only strings in TEXT column
blanks = []

# Iterate through the Series using items() instead of iteritems()
for index, text in df['text'].items():  # items() is the modern replacement
    if isinstance(text, str) and text.isspace():  # checks if it's a string and contains only whitespace
        blanks.append(index)  # adds index to empty list

len(blanks) #use to calculate number of white spaces.
print(f"Number of empty/whitespace-only texts: {len(blanks)}")

In [ ]:
blanks

In [ ]:
df['text'][10923]

In [ ]:
df.shape

In [ ]:
df.drop(blanks, inplace=True)

In [ ]:
df.shape

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy 
import re
nlp=spacy.load('en_core_web_sm')

# Stopwords 

A stop word is a commonly used word (such as 'the, as , a , in' )that a search engine is programmed to ignore, both when indexing entries for searching and when recieving as the result of a search query. We would not any of these words to take up space in our database, or taken up varriable processing time. For this, we can remove them easily by storing a list of words that you consider to stop words NLTK(Natural Language tool kit) in python has a list of stopwords stored in 16 different languages. You can find them in the nltk_data directory.

In [ ]:
 lemma=WordNetLemmatizer()

In [ ]:
#stopwords of spacy
list1=nlp.Defaults.stop_words
print(len(list1))

#stopwords of NLTK
list2=stopwords.words('english')
print(len(list2))

#combinig the stopword list
Stopwords=set((set(list1)|set(list2)))
print(len(Stopwords))

In [ ]:
def clean_text(text):

    string = ""
    #lower casing 
    text = text.lower()
    #simplifying text
    text=re.sub(r"i'm","i am",text)
    text=re.sub(r"he's","he is",text)
    text=re.sub(r"she's","she is",text)
    text=re.sub(r"that's","that is",text)
    text=re.sub(r"what's","what is",text)
    text=re.sub(r"where's","where is",text)
    text=re.sub(r"\'ll","will",text)
    text=re.sub(r"\'ve","have",text)
    text=re.sub(r"\'re","are",text)
    text=re.sub(r"\'d","would",text)
    text=re.sub(r"won't","will not",text)
    text=re.sub(r"can't","cannot",text)
    #removing any special character:
    text=re.sub(r"[-()\"#!@$%^&*{}?.,:]"," ",text)
    text=re.sub(r"\s+"," ",text)
    text=re.sub('[^A-Za-z0-9]+',' ',text)

    for word in text.split():
        if word not in Stopwords:
            string+=lemma.lemmatize(word)+" "

    return string

In [ ]:
df['text'][10]

In [ ]:
#Checking if the function work over a single text data before applying to the whole data set!!
clean_text(df['text'][10])

In [ ]:
#cleaning the whole data :
df['text']=df['text'].apply(clean_text)

In [ ]:
df['text']

In [ ]:
df['text'][20]

We can also clean the data in our ways like removing link from the news text, removing common words and different things we feel necessary for the crutial data clean dataset to perform fast and accurate.

# Word Cloud

the word with more presence will have higer frequency and vice versa.

In [ ]:
from wordcloud import WordCloud

In [ ]:
#True News
plt.figure(figsize = (20,20))
Wc = WordCloud(max_words= 500, width = 1600, height = 800).generate(" ".join(df[df.category == 0].text))
plt.axis("off")
plt.imshow(Wc, interpolation = 'bilinear')

In [ ]:
#pil is pillow and used for image manipulation
from PIL import Image

In [ ]:
#lets make the image work for thumbs-TRUE here and make it interesting
thumb='/kaggle/input/image-wordcloud/thumbs-up.png'
icon=Image.open(thumb)
mask=Image.new(mode="RGB", size=icon.size, color=(255,255,255))
mask.paste(icon, box=icon)

rgb_array=np.array(mask)

In [ ]:
#true news visual representation
plt.figure(figsize = (10,10))
Wc = WordCloud(mask=rgb_array, max_words= 2000, width = 1600, height = 800).generate(" ".join(df[df.category == 0].text))
plt.axis("off")
plt.imshow(Wc, interpolation = 'bilinear')

In [ ]:
#lets make the image work for skull-FAKE here and make it interesting
skull='/kaggle/input/image-wordcloud/skull-icon.png'
icon=Image.open(skull)
mask=Image.new(mode="RGB", size=icon.size, color=(255,255,255))
mask.paste(icon, box=icon)

rgb_array=np.array(mask)

In [ ]:
#fake news visual representation
plt.figure(figsize = (10,10))
Wc = WordCloud(mask=rgb_array, max_words= 2000, width = 1600, height = 800).generate(" ".join(df[df.category == 1].text))
plt.axis("off")
plt.imshow(Wc, interpolation = 'bilinear')

# MODEL BUILDING 

Machine learning algo only understands numerical data no text data we need to convert it.

In [ ]:
#it is pupil and packing in python 
from sklearn.model_selection import train_test_split

x=df["text"] #feature
y=df["category"] #target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#text data into numerical format - TFIDF vectoriser
# THIS IS BASICALLY DATA TRANSFORMATION
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
X_train_tfidf=vectorizer.fit_transform(X_train)

In [ ]:
X_train_tfidf

In [ ]:
# LINEAR SUPPPORT VECTOR CLACIFIER - LinearSVC
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

In [ ]:
# To avoid repetition of applying of vectorization over test and train data we use pipeline where we can do this simulataneos -
# PIPELINE = DATA TRANSFORMATION + LinearSVC together!!
from sklearn.pipeline import Pipeline
text_clf=Pipeline([("tfidf", TfidfVectorizer()),("clf", LinearSVC())])
text_clf.fit(X_train,y_train)

In [ ]:
# Now we are storing the prediction by calling the test case and storing in a varraible
predictions = text_clf.predict(X_test)

In [ ]:
# We are doing this to check how well our model performed on the basis of metrics
from sklearn import metrics
print(metrics.classification_report(y_test,predictions))

In [ ]:
print(metrics.accuracy_score(y_test,predictions))

In [ ]:
# Matrix representation
# { True_news Fake_news }
# { Fake_news True_news }

print(metrics.confusion_matrix(y_test,predictions))

In [ ]:
!pip install shap
import shap
import numpy as np

In [ ]:
# Extract the TF-IDF vectorizer from your pipeline
vectorizer = text_clf.named_steps["tfidf"]
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
model = text_clf.named_steps["clf"]

explainer = shap.Explainer(model, X_test_tfidf)
shap_values = explainer(X_test_tfidf[:5])  # You can increase number if needed

In [ ]:
import shap

# Extract components
model = text_clf.named_steps["clf"]
vectorizer = text_clf.named_steps["tfidf"]
X_test_tfidf = vectorizer.transform(X_test)

# Define masker
masker = shap.maskers.Independent(X_test_tfidf)

# Create explainer
explainer = shap.LinearExplainer(model, masker=masker)

# Get SHAP values
shap_values = explainer(X_test_tfidf[:5])

In [ ]:
shap.summary_plot(shap_values.values, X_test_tfidf[:5], feature_names=vectorizer.get_feature_names_out())

In [ ]:
import joblib

# Save the pipeline (text_clf = your trained Pipeline)
joblib.dump(text_clf, 'fake_news_model.pkl')

In [ ]:
joblib.dump(text_clf, 'fake_news_model.pkl')

In [ ]:
df[df["category"] == 0]["text"].iloc[0]

In [ ]:
print(df[df["category"] == 0]["text"].sample(1).values[0])